<a href="https://colab.research.google.com/github/hiwon-lee/openAI_RAG/blob/master/blueAI_langchain_rag_chroma_gpt_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assistant + tools
- **code_interpreter**
   - 토큰 길이에 상관 없이, 코드를 생성해서 답변할 수 있는 도구 입니다.
   - 예: 피보나치 수열을 만들어줘 -> 피보나치 수열을 생성하는 함수를 생성한 후 이것의 실행 결과로 답변을 만듦 (cf., Code Interpreter가 아니라면, text 기반의 sequence generation으로 답변을 함)
   - 함수를 생성하는 방식이기 때문에, input, output의 token length 제약을 벗어나서 활용을 할 수 있습니다.
   - 데이터와 그래프 이미지가 포함된 파일을 생성할 수 있습니다.
- **[이번 내용] file_search**
   - 우리가 업로드한 자료에서 검색한 이후 답변을 만들어 줍니다. (RAG를 활용하는 것과 비슷)
   - Foundation model을 sFT하는데 걸리는 시간을 생각하면, 대부분의 짧은 배치는 RAG 형태로 구성하고, 긴배치(ex., 1~2주일에 한번)는 sFT를 하는 방식이 적절할 것으로 보입니다.
   - File을 Vector Store에 넣어두고 Assistant가 이를 활용할 수 있습니다.
   - 파일 하나당 최대 크기 512MB, 최대 토큰수 5,000,000까지 지원합니다.
   - [지원하는 파일 타입](https://platform.openai.com/docs/assistants/tools/file-search/supported-files)
   - [Known Limitation](https://platform.openai.com/docs/assistants/tools/file-search/how-it-works)
   - 아직 이미지 파일이나, csv, jsonl을 지원하지 않는데 추후 업데이트 될 것으로 보입니다. (24-05-21 기준)
- **function**
   - 우리가 직접 만든 함수를 사용할 수 있게 합니다.

\

## 기본 작업
- Open AI 라이브러리 설치
- Dotenv 설치
- Google Drive 연결
- API Key 로드
- Open AI 클라이언트 객체 생성

In [ ]:
# 실행을 위한 기본 설치 및 로드
!pip install --upgrade openai -q
!pip show openai | grep Version
!pip install -U python-dotenv PyPDF langchain langchain-community -q
!pip install pypdf sentence-transformers chromadb -q
!pip install langchain_chroma langchain_openai beautifulsoup4 chromadb gradio -q


from dotenv import load_dotenv
import json
from openai import OpenAI
import os
from google.colab import drive

Version: 1.41.1


In [ ]:
drive.mount('/content/drive', force_remount=True)

load_dotenv(
    dotenv_path='drive/MyDrive/AUTH/.env',
    verbose=True,
)
api_key = os.environ.get("OPEN_API_KEY")
client = OpenAI(api_key=api_key)

# 객체 내용 출력 유틸
def show_json(obj):
  display(json.loads(obj.model_dump_json()))

def _get_response(thread_id):
    return client.beta.threads.messages.list(thread_id=thread_id, order="asc")

# Thread message 출력 유틸
def print_message(thread_id):
    for res in _get_response(thread_id):
        print(f"[{res.role.upper()}]\n{res.content[0].text.value}\n")

Mounted at /content/drive


## File-Search Assistant 생성

## Upload files and add them to a Vector Store
- Vector store를 하나 만들고 파일을 업로드 한 후 모든 파일의 업로드 상태가 종료되었는지 확인해야 합니다.

In [ ]:
# 폴더 전체로 넣을 경우
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

DATA_PATH = "/content/drive/MyDrive/ragPrompt"
document_loader = PyPDFDirectoryLoader(DATA_PATH)
documents = document_loader.load()
print(documents)



[Document(metadata={'source': '/content/drive/MyDrive/ragPrompt/blueAI prompt.pdf', 'page': 0}, page_content='03. blueAI xaml\nsome examples are given for knowing how to make xaml format\n몇가지  액티비티를  알려줄테니  잘  기억하고  활용하세요 .\nxaml 에  대한  것이  아닌  어떠한  말도  해서는  안되며  요구사항을  만족시키는  xaml 을  정확히  출력해야합니다 .\n위의 예시와  같이  활용하면  됩니다 .\nDoubleClick 과  PostW ait 과  같은  변수를  유동적으로  잘  활용하세요\n예시 설명 : xml 상의  선택된  요소를  사전에  기다리지  않고 (postW ait 이  0 초이기  때문 ) 클릭하는  xml\n사용자 : 변수  url 에  구글  url 을  할당한  후에  chrome 을  통해  해당  url 에  접근한  5 초  뒤에  더블  클릭을  해\n시스템 :이름: OpenUrl \n용도: 입력된 Browser 를  통해  Url 을  여는  액티비티  \nxml format: <bn:OpenURL NewTab= \\"{x:Null}\\" UserDataFolderMode= \\"{x:Null}\\" UserDataFolderPath= \\"\n{x:Null}\\" Browser= \\"chrome\\" DisplayName= \\" 잡코리아  접속 \\" sap2010:WorkflowViewState.IdRef= \\"OpenURL_2\\"\nUrl= \\"https://www.jobkorea.co.kr/\\" refresh= \\"False\\" /> \n필수 변수: Url 특이  사항 : DisplayName 은  유저에게  보이는  이름으로  시스템이  판단하여  용도에  따른  이름을  입력하기  바람  \n예시 설명: chrome bro

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document


def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size = 500,
    #     chunk_overlap = 0
    # )
    return text_splitter.split_documents(documents)

chunks = split_documents(documents)
chunks[0]

Document(metadata={'source': '/content/drive/MyDrive/ragPrompt/blueAI prompt.pdf', 'page': 0}, page_content='03. blueAI xaml\nsome examples are given for knowing how to make xaml format\n몇가지  액티비티를  알려줄테니  잘  기억하고  활용하세요 .\nxaml 에  대한  것이  아닌  어떠한  말도  해서는  안되며  요구사항을  만족시키는  xaml 을  정확히  출력해야합니다 .\n위의 예시와  같이  활용하면  됩니다 .\nDoubleClick 과  PostW ait 과  같은  변수를  유동적으로  잘  활용하세요\n예시 설명 : xml 상의  선택된  요소를  사전에  기다리지  않고 (postW ait 이  0 초이기  때문 ) 클릭하는  xml\n사용자 : 변수  url 에  구글  url 을  할당한  후에  chrome 을  통해  해당  url 에  접근한  5 초  뒤에  더블  클릭을  해\n시스템 :이름: OpenUrl \n용도: 입력된 Browser 를  통해  Url 을  여는  액티비티  \nxml format: <bn:OpenURL NewTab= \\"{x:Null}\\" UserDataFolderMode= \\"{x:Null}\\" UserDataFolderPath= \\"\n{x:Null}\\" Browser= \\"chrome\\" DisplayName= \\" 잡코리아  접속 \\" sap2010:WorkflowViewState.IdRef= \\"OpenURL_2\\"\nUrl= \\"https://www.jobkorea.co.kr/\\" refresh= \\"False\\" />')

vector db로 임베딩

In [ ]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

from langchain.vectorstores import Chroma
vector_database = Chroma.from_documents(documents = chunks, embedding = embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI  # ChatOpenAI를 사용할 경우
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(api_key=api_key, model_name="gpt-4o", temperature=0)
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)


prompt =   r"""당신은 blueAI의 xaml 제작기 이며, 만든 xaml은 uipath와 같은 환경에서 rpa를 동작하는데에 활용될 것입니다.
그러나 지금 xaml이 활용될 곳은 bebot이라는 rpa서비스 이므로, xaml의 형식은 uipath와 달라야합니다.
형식은 당신 마음대로 제작할 수 없습니다.
vectordb에서 context가 당신에게 주어질 것이고, 당신은 그 형식에 맞춰 xaml을 제작해야합니다. 절대 임의로 제작해서는 안됩니다.
xaml은 가장 효율적인 방식으로 제작되어야 합니다.
다음 몇가지 예시를 보여드리겠습니다.

user :'네이버를 열어줘'
system :
<Sequence>
  <bn:OpenURL NewTab= "{{x:Null}}" UserDataFolderMode= "{{x:Null}}" UserDataFolderPath= "{{x:Null}}" Browser= "chrome" DisplayName= "네이버 접속" sap2010:WorkflowViewState.IdRef= "OpenURL_1" Url= "https://www.naver.com/" refresh= "False" />
</Sequence>

user : '국토교통부 사이트에서 주택청약 관련 PDF 파일을 다운로드하고, 이를 hiiwonii1012@gmail.com주소의 이메일로 전송해줘'
system :
<Sequence DisplayName="국토교통부" sap2010:WorkflowViewState.IdRef="Sequence_2">
    <Sequence.Variables>
      <Variable x:TypeArguments="bn:NMElement" Name="item" />
      <Variable x:TypeArguments="x:String" Name="name" />
      <Variable x:TypeArguments="x:String" Default="C:\Users\USER\Downloads" Name="baseurl" />
    </Sequence.Variables>
    <bn:OpenURL Browser="{{x:Null}}" NewTab="{{x:Null}}" UserDataFolderMode="{{x:Null}}" UserDataFolderPath="{{x:Null}}" sap2010:WorkflowViewState.IdRef="OpenURL_1" Url="https://www.molit.go.kr/portal.do" refresh="False" />
    <bn:GetElement Elements="{{x:Null}}" From="{{x:Null}}" Image="{{x:Null}}" LoopAction="{{x:Null}}" Timeout="{{x:Null}}" WaitForReady="{{x:Null}}" sap2010:WorkflowViewState.IdRef="GetElement_2" MaxResults="1" MinResults="1" Selector="%[
  {{
    "Selector": "NM",
    "browser": "chrome",
    "frame": "-1",
    "url": ""
  }},
  {{
    "xpath": "//nav[@id=\&quot;gnb\&quot;]/ul/li[3]/a/i",
    "cssselector": "html > body > div.main > header > div > div.wrap-header-cont > div > div.wrap-gnb > div > nav > ul > li:nth-child(3) > a > i",
    "tagname": "I",
    "Text": "정책자료"
  }}]">
      <bn:GetElement.Variables>
        <Variable x:TypeArguments="x:Int32" Default="0" Name="Index" />
        <Variable x:TypeArguments="x:Int32" Default="0" Name="Total" />
      </bn:GetElement.Variables>
      <ActivityAction x:TypeArguments="bn:NMElement">
        <ActivityAction.Argument>
          <DelegateInArgument x:TypeArguments="bn:NMElement" Name="item" />
        </ActivityAction.Argument>
        <ba:ClickElement KeyModifiers="{{x:Null}}" AnimateMouse="False" Button="1" DoubleClick="False" Element="[item]" Focus="False" sap2010:WorkflowViewState.IdRef="ClickElement_2" OffsetX="5" OffsetY="5" PostWait="00:00:00" VirtualClick="True" />
      </ActivityAction>
    </bn:GetElement>
    <bn:GetElement Elements="{{x:Null}}" From="{{x:Null}}" Image="{{x:Null}}" LoopAction="{{x:Null}}" Timeout="{{x:Null}}" WaitForReady="{{x:Null}}" sap2010:WorkflowViewState.IdRef="GetElement_3" MaxResults="1" MinResults="1" Selector="%[
  {{
    "Selector": "NM",
    "browser": "chrome",
    "frame": "-1",
    "url": ""
  }},
  {{
    "xpath": "//input[@id=\&quot;search\&quot;]",
    "cssselector": "html > body > div.wrap > section > div.wrap-content > article > form > div:nth-child(3) > fieldset > div.sear_word > div.sear_word_txt > input",
    "id": "search",
    "Name": "search",
    "type": "text",
    "tagname": "INPUT",
    "title": "주제어입력"
  }}]">
      <bn:GetElement.Variables>
        <Variable x:TypeArguments="x:Int32" Default="0" Name="Index" />
        <Variable x:TypeArguments="x:Int32" Default="0" Name="Total" />
      </bn:GetElement.Variables>
      <ActivityAction x:TypeArguments="bn:NMElement">
        <ActivityAction.Argument>
          <DelegateInArgument x:TypeArguments="bn:NMElement" Name="item" />
        </ActivityAction.Argument>
        <Assign sap2010:WorkflowViewState.IdRef="Assign_2">
          <Assign.To>
            <OutArgument x:TypeArguments="x:String">[item.Value]</OutArgument>
          </Assign.To>
          <Assign.Value>
            <InArgument x:TypeArguments="x:String">주택청약</InArgument>
          </Assign.Value>
        </Assign>
      </ActivityAction>
    </bn:GetElement>
    <bn:GetElement Elements="{{x:Null}}" From="{{x:Null}}" Image="{{x:Null}}" LoopAction="{{x:Null}}" Timeout="{{x:Null}}" WaitForReady="{{x:Null}}" sap2010:WorkflowViewState.IdRef="GetElement_4" MaxResults="1" MinResults="1" Selector="%[
  {{
    "Selector": "NM",
    "browser": "chrome",
    "frame": "-1",
    "url": ""
  }},
  {{
    "xpath": "//article[@id=\&quot;cont-body\&quot;]/form/div[1]/fieldset/div[3]/div[2]/button/i",
    "cssselector": "html > body > div.wrap > section > div.wrap-content > article > form > div:nth-child(3) > fieldset > div.sear_word > div.sear_word_txt > button > i",
    "tagname": "I",
    "Text": "검색"
  }}]">
      <bn:GetElement.Variables>
        <Variable x:TypeArguments="x:Int32" Default="0" Name="Index" />
        <Variable x:TypeArguments="x:Int32" Default="0" Name="Total" />
      </bn:GetElement.Variables>
      <ActivityAction x:TypeArguments="bn:NMElement">
        <ActivityAction.Argument>
          <DelegateInArgument x:TypeArguments="bn:NMElement" Name="item" />
        </ActivityAction.Argument>
        <ba:ClickElement KeyModifiers="{{x:Null}}" AnimateMouse="False" Button="1" DoubleClick="False" Element="[item]" Focus="False" sap2010:WorkflowViewState.IdRef="ClickElement_3" OffsetX="5" OffsetY="5" PostWait="00:00:00" VirtualClick="True" />
      </ActivityAction>
    </bn:GetElement>
    <Delay Duration="00:00:02" sap2010:WorkflowViewState.IdRef="Delay_1" />
    <bn:GetElement Elements="{{x:Null}}" From="{{x:Null}}" Image="{{x:Null}}" LoopAction="{{x:Null}}" Timeout="{{x:Null}}" WaitForReady="{{x:Null}}" sap2010:WorkflowViewState.IdRef="GetElement_5" MaxResults="1" MinResults="1" Selector="%[
  {{
    "Selector": "NM",
    "browser": "chrome",
    "frame": "-1",
    "url": ""
  }},
  {{
    "xpath": "//table[@id=\&quot;board-list\&quot;]/tbody/tr/td[2]/a",
    "cssselector": "html > body > div.wrap > section > div.wrap-content > article > table > tbody > tr > td.bd_title > a",
    "tagname": "A",
    "Text": "주택청약 FAQ",
    "href": "dtl.jsp?search=주택청약&amp;srch_dept_nm=&amp;srch_dept_id=&amp;srch_usr_nm=&amp;srch_usr_titl=Y&amp;srch_usr_ctnt=&amp;search_regdate_s=&amp;search_regdate_e=&amp;psize=10&amp;s_category=&amp;p_category=&amp;lcmspage=1&amp;id=4765"
  }}]">
      <bn:GetElement.Variables>
        <Variable x:TypeArguments="x:Int32" Default="0" Name="Index" />
        <Variable x:TypeArguments="x:Int32" Default="0" Name="Total" />
      </bn:GetElement.Variables>
      <ActivityAction x:TypeArguments="bn:NMElement">
        <ActivityAction.Argument>
          <DelegateInArgument x:TypeArguments="bn:NMElement" Name="item" />
        </ActivityAction.Argument>
        <ba:ClickElement KeyModifiers="{{x:Null}}" AnimateMouse="False" Button="1" DoubleClick="False" Element="[item]" Focus="False" sap2010:WorkflowViewState.IdRef="ClickElement_4" OffsetX="5" OffsetY="5" PostWait="00:00:00" VirtualClick="True" />
      </ActivityAction>
    </bn:GetElement>
    <Delay Duration="00:00:02" sap2010:WorkflowViewState.IdRef="Delay_2" />
    <bn:GetElement Elements="{{x:Null}}" From="{{x:Null}}" Image="{{x:Null}}" LoopAction="{{x:Null}}" Timeout="{{x:Null}}" WaitForReady="{{x:Null}}" sap2010:WorkflowViewState.IdRef="GetElement_6" MaxResults="1" MinResults="1" Selector="%[
  {{
    "Selector": "NM",
    "browser": "chrome",
    "frame": "-1",
    "url": "https://www.molit.go.kr/USR/policyData/m_34681/dtl.jsp?search=주택청약&amp;srch_dept_nm=&amp;srch_dept_id=&amp;srch_usr_nm=&amp;srch_usr_titl=Y&amp;srch_usr_ctnt=&amp;search_regdate_s=&amp;search_regdate_e=&amp;psize=10&amp;s_category=&amp;p_category=&amp;lcmspage=1&amp;id=4765"
  }},
  {{
    "xpath": "//article[@id=\&quot;cont-body\&quot;]/div[1]/ul/li[7]/span/a",
    "cssselector": "html > body > div.wrap > section > div.wrap-content > article > div.bd_view > ul > li.file > span > a",
    "tagname": "A",
    "Text": "★ 2024 주택청약 FAQ.pdf",
    "href": "/portal/common/download/DownloadMltm2.jsp?FilePath=portal/DextUpload/202405/20240529_165121_948.pdf&amp;FileName=★ 2024 주택청약 FAQ.pdf"
  }}]">
      <bn:GetElement.Variables>
        <Variable x:TypeArguments="x:Int32" Default="0" Name="Index" />
        <Variable x:TypeArguments="x:Int32" Default="0" Name="Total" />
      </bn:GetElement.Variables>
      <ActivityAction x:TypeArguments="bn:NMElement">
        <ActivityAction.Argument>
          <DelegateInArgument x:TypeArguments="bn:NMElement" Name="item" />
        </ActivityAction.Argument>
        <Sequence sap2010:WorkflowViewState.IdRef="Sequence_4">
          <ba:ClickElement KeyModifiers="{{x:Null}}" AnimateMouse="False" Button="1" DoubleClick="False" Element="[item]" Focus="False" sap2010:WorkflowViewState.IdRef="ClickElement_5" OffsetX="5" OffsetY="5" PostWait="00:00:00" VirtualClick="True" />
          <Assign sap2010:WorkflowViewState.IdRef="Assign_3">
            <Assign.To>
              <OutArgument x:TypeArguments="x:String">[name]</OutArgument>
            </Assign.To>
            <Assign.Value>
              <InArgument x:TypeArguments="x:String">[item.Value]</InArgument>
            </Assign.Value>
          </Assign>
        </Sequence>
      </ActivityAction>
    </bn:GetElement>
    <Delay Duration="00:00:03" sap2010:WorkflowViewState.IdRef="Delay_3" />
    <bm:SendMailUsingSMTP Body="{{x:Null}}" DeliveryMessage="{{x:Null}}" HiddenReference="{{x:Null}}" Reference="{{x:Null}}" ReplyEmail="{{x:Null}}" SenderEmailDisplaied="{{x:Null}}" SenderName="{{x:Null}}" StatusCode="{{x:Null}}" Attachments="[baseurl + \"+name]" sap2010:WorkflowViewState.IdRef="SendMailUsingSMTP_1" ReceiverEmail="hiiwonii1012@gmail.com" SecureConnection="1" SenderEmail="kwakkyoleen@outlook.kr" SenderPassword="0312gk**" SmtpServer="0" SmtpServerPort="587" SmtpServerUrl="smtp.office365.com" Subject="데이터 수집 완료 되었습니다" bodyIsHtml="False" ignoreCrl="False" />
    <sads1:DebugSymbol.Symbol>dw1DOlxFbXB0eS54YW1sAA==</sads1:DebugSymbol.Symbol>
</Sequence>

와 같이 대답해야합니다.
xaml은 sequence 태그로 감싸져야합니다.

Context:
{context}
---

user: {question}
system:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)
# question  ="네이버에서 사과를 검색해줘"





In [ ]:
!pip install halo -q

In [ ]:
import gradio as gr
from bs4 import BeautifulSoup as bs
import pprint
from halo import Halo

xmlresult = ""
def gpt_llm(query):
  # Retrieval QA
  qa_chain = RetrievalQA.from_chain_type(
      llm=llm,
      retriever=vector_database.as_retriever(),
      chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
  )

  retrieved_docs = vector_database.similarity_search(query, k=7)  # 가장 관련성 높은 문서 1개 검색

  # 검색된 문서들에서 context 결합하기
  if retrieved_docs:
      context = "\n\n".join([doc.page_content for doc in retrieved_docs])  # 3개의 문서 내용을 결합
      # print("Retrieved Context:", context)
  else:
      print("No relevant context found.")


  # 입력값을 "context"와 "x"로 묶어서 전달
  input_data = {
      "context": context,
      "query": query
  }

  result = qa_chain(input_data)

  # print(result["result"])

  return result["result"]


def generate_response(messages):
    # Create a loading spinner
    spinner = Halo(text='Loading...', spinner='dots')
    spinner.start()

# Define the RAG setup
retriever = vector_database.as_retriever()

def rag_chain(query):
    retrieved_docs = retriever.invoke(query)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    # return gpt_llm(question, formatted_context)
    return gpt_llm(query)

# Define the Gradio interface
def get_important_facts(query):
    xmlresult = rag_chain(query)
    # print(xmlresult)
    return xmlresult


# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
  outputs="text",
  title="blueAI",
  description="Ask questions about the proveded context",
)

# Launch the Gradio app
# iface.launch()
question = input()
xmlresult = get_important_facts(question)
xmlresult


open naver


'<Sequence>\n  <bn:OpenURL NewTab="{x:Null}" UserDataFolderMode="{x:Null}" UserDataFolderPath="{x:Null}" Browser="chrome" DisplayName="네이버 접속" sap2010:WorkflowViewState.IdRef="OpenURL_1" Url="https://www.naver.com/" refresh="False" />\n</Sequence>'

# 여기까지가 xaml을 출력하는 부 입니다.

v7에서는 해당 xaml을 json형식에 맞게 가공하는 절차가 추가됩니다.

In [ ]:
xmlresult

'<Sequence>\n  <bn:OpenURL NewTab="{x:Null}" UserDataFolderMode="{x:Null}" UserDataFolderPath="{x:Null}" Browser="chrome" DisplayName="네이버 접속" sap2010:WorkflowViewState.IdRef="OpenURL_1" Url="https://www.naver.com/" refresh="False" />\n</Sequence>'

In [ ]:
# 이스케이프 처리를 위한 백슬래시 추가
escaped_xml = xmlresult.replace('"', r'\"')

# 결과 출력
print(escaped_xml)

<Sequence>
  <bn:OpenURL NewTab=\"{x:Null}\" UserDataFolderMode=\"{x:Null}\" UserDataFolderPath=\"{x:Null}\" Browser=\"chrome\" DisplayName=\"네이버 접속\" sap2010:WorkflowViewState.IdRef=\"OpenURL_1\" Url=\"https://www.naver.com/\" refresh=\"False\" />
</Sequence>


In [ ]:
template = r"""{{
  "queue": null,
  "Xaml": "<Activity mc1:Ignorable=\"sads sap sap2010\" x:Class=\"openurl_uipath.xaml\" sap2010:WorkflowViewState.IdRef=\"새_워크플로우_1\" mva1:VisualBasic.Settings=\"Assembly references and imported namespaces serialized as XML namespaces\" xmlns=\"http://schemas.microsoft.com/netfx/2009/xaml/activities\" xmlns:b=\"clr-namespace:beBOT;assembly=beBOT\" xmlns:b1=\"clr-namespace:beBOT;assembly=beBOT.Interfaces\" xmlns:ba=\"clr-namespace:beBOT.Activities;assembly=beBOT\" xmlns:bi=\"clr-namespace:beBOT.Image;assembly=beBOT.Image\" xmlns:bd=\"clr-namespace:beBOT.Database;assembly=beBOT.Database\" xmlns:bi1=\"clr-namespace:beBOT.Interfaces;assembly=beBOT.Interfaces\" xmlns:bi2=\"clr-namespace:beBOT.Interfaces;assembly=beBOT.Interfaces\" xmlns:bia=\"clr-namespace:beBOT.Interfaces.Activities;assembly=beBOT.Interfaces\" xmlns:bie=\"clr-namespace:beBOT.Interfaces.entity;assembly=beBOT.Interfaces\" xmlns:bii=\"clr-namespace:beBOT.Interfaces.IPCService;assembly=beBOT.Interfaces\" xmlns:bii1=\"clr-namespace:beBOT.Interfaces.Input;assembly=beBOT.Interfaces\" xmlns:bii2=\"clr-namespace:beBOT.Interfaces.Image;assembly=beBOT.Interfaces\" xmlns:bim=\"clr-namespace:beBOT.Interfaces.mq;assembly=beBOT.Interfaces\" xmlns:bio=\"clr-namespace:beBOT.Interfaces.Overlay;assembly=beBOT.Interfaces\" xmlns:bir=\"clr-namespace:beBOT.Interfaces.Resources;assembly=beBOT.Interfaces\" xmlns:bis=\"clr-namespace:beBOT.Interfaces.Selector;assembly=beBOT.Interfaces\" xmlns:biv=\"clr-namespace:beBOT.Interfaces.VT;assembly=beBOT.Interfaces\" xmlns:biv1=\"clr-namespace:beBOT.Interfaces.Views;assembly=beBOT.Interfaces\" xmlns:biw=\"clr-namespace:beBOT.Interfaces.win32;assembly=beBOT.Interfaces\" xmlns:bn=\"clr-namespace:beBOT.NM;assembly=beBOT.NM\" xmlns:bna=\"clr-namespace:beBOT.NM.Activities;assembly=beBOT.NM\" xmlns:bnp=\"clr-namespace:beBOT.NM.pipe;assembly=beBOT.NM\" xmlns:bnr=\"clr-namespace:beBOT.NM.Resources;assembly=beBOT.NM\" xmlns:bns=\"clr-namespace:beBOT.NM.Snippets;assembly=beBOT.NM\" xmlns:bnv=\"clr-namespace:beBOT.NM.Views;assembly=beBOT.NM\" xmlns:br=\"clr-namespace:beBOT.Resources;assembly=beBOT\" xmlns:bs=\"clr-namespace:beBOT.Store;assembly=beBOT\" xmlns:bv=\"clr-namespace:beBOT.Views;assembly=beBOT\" xmlns:bv1=\"clr-namespace:beBOT.ViewModel;assembly=beBOT\" xmlns:bw=\"clr-namespace:beBOT.WorkItems;assembly=beBOT\" xmlns:boa=\"clr-namespace:beBOT.Office.Activities;assembly=beBOT.Office\" xmlns:bu=\"clr-namespace:beBOT.Utilities;assembly=beBOT.Utilities\" xmlns:bm=\"clr-namespace:beBOT.Mail;assembly=beBOT.Mail\" xmlns:bi3=\"clr-namespace:beBOT.Image;assembly=beBOT.Image\" xmlns:c=\"clr-namespace:Costura;assembly=OpenRPA.NM\" xmlns:m=\"clr-namespace:ManagedSSPI;assembly=System.Net.WebSockets.Client.Managed\" xmlns:mc=\"clr-namespace:Microsoft.CSharp;assembly=System\" xmlns:mc1=\"http://schemas.openxmlformats.org/markup-compatibility/2006\" xmlns:mss=\"clr-namespace:Microsoft.SqlServer.Server;assembly=System.Data\" xmlns:mv=\"clr-namespace:Microsoft.VisualBasic;assembly=System\" xmlns:mv1=\"clr-namespace:Microsoft.VisualBasic;assembly=Microsoft.VisualBasic\" xmlns:mva=\"clr-namespace:Microsoft.VisualBasic.ApplicationServices;assembly=Microsoft.VisualBasic\" xmlns:mva1=\"clr-namespace:Microsoft.VisualBasic.Activities;assembly=System.Activities\" xmlns:mvc=\"clr-namespace:Microsoft.VisualBasic.CompilerServices;assembly=Microsoft.VisualBasic\" xmlns:mvd=\"clr-namespace:Microsoft.VisualBasic.Devices;assembly=Microsoft.VisualBasic\" xmlns:mvf=\"clr-namespace:Microsoft.VisualBasic.FileIO;assembly=Microsoft.VisualBasic\" xmlns:mvl=\"clr-namespace:Microsoft.VisualBasic.Logging;assembly=Microsoft.VisualBasic\" xmlns:mvm=\"clr-namespace:Microsoft.VisualBasic.MyServices;assembly=Microsoft.VisualBasic\" xmlns:mvmi=\"clr-namespace:Microsoft.VisualBasic.MyServices.Internal;assembly=Microsoft.VisualBasic\" xmlns:mw=\"clr-namespace:Microsoft.Win32;assembly=mscorlib\" xmlns:mw1=\"clr-namespace:Microsoft.Win32;assembly=System\" xmlns:mws=\"clr-namespace:Microsoft.Win32.SafeHandles;assembly=mscorlib\" xmlns:mws1=\"clr-namespace:Microsoft.Win32.SafeHandles;assembly=System.Core\" xmlns:mws2=\"clr-namespace:Microsoft.Win32.SafeHandles;assembly=System\" xmlns:o=\"clr-namespace:OpenRPA;assembly=OpenRPA\" xmlns:o1=\"clr-namespace:OpenRPA;assembly=OpenRPA.Interfaces\" xmlns:ou=\"clr-namespace:OpenRPA.Utilities;assembly=OpenRPA.Utilities\" xmlns:oa=\"clr-namespace:OpenRPA.Activities;assembly=OpenRPA\" xmlns:oi=\"clr-namespace:OpenRPA.Image;assembly=OpenRPA.Image\" xmlns:oi1=\"clr-namespace:OpenRPA.Interfaces;assembly=OpenRPA.Interfaces\" xmlns:oi2=\"clr-namespace:OpenRPA.IE;assembly=OpenRPA.IE\" xmlns:oia=\"clr-namespace:OpenRPA.Interfaces.Activities;assembly=OpenRPA.Interfaces\" xmlns:oie=\"clr-namespace:OpenRPA.Interfaces.entity;assembly=OpenRPA.Interfaces\" xmlns:oii=\"clr-namespace:OpenRPA.Interfaces.IPCService;assembly=OpenRPA.Interfaces\" xmlns:oii1=\"clr-namespace:OpenRPA.Interfaces.Input;assembly=OpenRPA.Interfaces\" xmlns:oii2=\"clr-namespace:OpenRPA.Interfaces.Image;assembly=OpenRPA.Interfaces\" xmlns:oim=\"clr-namespace:OpenRPA.Interfaces.mq;assembly=OpenRPA.Interfaces\" xmlns:oio=\"clr-namespace:OpenRPA.Interfaces.Overlay;assembly=OpenRPA.Interfaces\" xmlns:oir=\"clr-namespace:OpenRPA.Interfaces.Resources;assembly=OpenRPA.Interfaces\" xmlns:ois=\"clr-namespace:OpenRPA.Interfaces.Selector;assembly=OpenRPA.Interfaces\" xmlns:oiv=\"clr-namespace:OpenRPA.Interfaces.Views;assembly=OpenRPA.Interfaces\" xmlns:oiw=\"clr-namespace:OpenRPA.Interfaces.win32;assembly=OpenRPA.Interfaces\" xmlns:on=\"clr-namespace:OpenRPA.NM;assembly=OpenRPA.NM\" xmlns:ona=\"clr-namespace:OpenRPA.NM.Activities;assembly=OpenRPA.NM\" xmlns:onp=\"clr-namespace:OpenRPA.NM.pipe;assembly=OpenRPA.NM\" xmlns:onr=\"clr-namespace:OpenRPA.NM.Resources;assembly=OpenRPA.NM\" xmlns:ons=\"clr-namespace:OpenRPA.NM.Snippets;assembly=OpenRPA.NM\" xmlns:onv=\"clr-namespace:OpenRPA.NM.Views;assembly=OpenRPA.NM\" xmlns:oo=\"clr-namespace:OpenRPA.OpenFlowDB;assembly=OpenRPA.OpenFlowDB\" xmlns:ooa=\"clr-namespace:OpenRPA.Office.Activities;assembly=OpenRPA.Office\" xmlns:or=\"clr-namespace:OpenRPA.Resources;assembly=OpenRPA\" xmlns:os=\"clr-namespace:OpenRPA.Store;assembly=OpenRPA\" xmlns:ov=\"clr-namespace:OpenRPA.Views;assembly=OpenRPA\" xmlns:ov1=\"clr-namespace:OpenRPA.ViewModel;assembly=OpenRPA\" xmlns:ow=\"clr-namespace:OpenRPA.WorkItems;assembly=OpenRPA\" xmlns:p=\"clr-namespace:XamlGeneratedNamespace;assembly=OpenRPA\" xmlns:s=\"clr-namespace:System;assembly=mscorlib\" xmlns:s1=\"clr-namespace:System;assembly=System.Core\" xmlns:s2=\"clr-namespace:System;assembly=System\" xmlns:s3=\"clr-namespace:System;assembly=System.Memory\" xmlns:s4=\"clr-namespace:System;assembly=Microsoft.Bcl.AsyncInterfaces\" xmlns:s5=\"clr-namespace:System;assembly=Microsoft.Bcl.HashCode\" xmlns:s6=\"clr-namespace:System;assembly=System.ValueTuple\" xmlns:s7=\"clr-namespace:System;assembly=System.ServiceModel\" xmlns:s8=\"clr-namespace:System;assembly=System.ComponentModel.Composition\" xmlns:s9=\"clr-namespace:System;assembly=System.Net.WebSockets.Client.Managed\" xmlns:sads=\"http://schemas.microsoft.com/netfx/2010/xaml/activities/debugger\" xmlns:sap=\"http://schemas.microsoft.com/netfx/2009/xaml/activities/presentation\" xmlns:sap2010=\"http://schemas.microsoft.com/netfx/2010/xaml/activities/presentation\" xmlns:sb=\"clr-namespace:System.Buffers;assembly=System.Memory\" xmlns:sbb=\"clr-namespace:System.Buffers.Binary;assembly=System.Memory\" xmlns:sbt=\"clr-namespace:System.Buffers.Text;assembly=System.Memory\" xmlns:sc=\"clr-namespace:System.Collections;assembly=mscorlib\" xmlns:sc1=\"clr-namespace:System.Configuration;assembly=System\" xmlns:sc2=\"clr-namespace:System.ComponentModel;assembly=System\" xmlns:sc3=\"clr-namespace:System.CodeDom;assembly=System\" xmlns:sca=\"clr-namespace:System.Configuration.Assemblies;assembly=mscorlib\" xmlns:scc=\"clr-namespace:System.Collections.Concurrent;assembly=mscorlib\" xmlns:scc1=\"clr-namespace:System.Collections.Concurrent;assembly=System\" xmlns:scc2=\"clr-namespace:System.CodeDom.Compiler;assembly=System\" xmlns:scc3=\"clr-namespace:System.ComponentModel.Composition;assembly=System.ComponentModel.Composition\" xmlns:scch=\"clr-namespace:System.ComponentModel.Composition.Hosting;assembly=System.ComponentModel.Composition\" xmlns:sccp=\"clr-namespace:System.ComponentModel.Composition.Primitives;assembly=System.ComponentModel.Composition\" xmlns:sccr=\"clr-namespace:System.ComponentModel.Composition.ReflectionModel;assembly=System.ComponentModel.Composition\" xmlns:scd=\"clr-namespace:System.ComponentModel.Design;assembly=System\" xmlns:scds=\"clr-namespace:System.ComponentModel.Design.Serialization;assembly=System\" xmlns:scg=\"clr-namespace:System.Collections.Generic;assembly=mscorlib\" xmlns:scg1=\"clr-namespace:System.Collections.Generic;assembly=System.Core\" xmlns:scg2=\"clr-namespace:System.Collections.Generic;assembly=System\" xmlns:scg3=\"clr-namespace:System.Collections.Generic;assembly=Microsoft.Bcl.AsyncInterfaces\" xmlns:scg4=\"clr-namespace:System.Collections.Generic;assembly=System.ServiceModel\" xmlns:sci=\"clr-namespace:System.Collections.Immutable;assembly=System.Collections.Immutable\" xmlns:sco=\"clr-namespace:System.Collections.ObjectModel;assembly=mscorlib\" xmlns:sco1=\"clr-namespace:System.Collections.ObjectModel;assembly=System\" xmlns:scs=\"clr-namespace:System.Collections.Specialized;assembly=System\" xmlns:sd=\"clr-namespace:System.Data;assembly=System.Data\" xmlns:sd1=\"clr-namespace:System.Dynamic;assembly=System.Core\" xmlns:sd2=\"clr-namespace:System.Diagnostics;assembly=System.Core\" xmlns:sd3=\"clr-namespace:System.Diagnostics;assembly=System\" xmlns:sd4=\"clr-namespace:System.Data;assembly=System.Data\" xmlns:sdc=\"clr-namespace:System.Diagnostics.Contracts;assembly=mscorlib\" xmlns:sdc1=\"clr-namespace:System.Diagnostics.CodeAnalysis;assembly=mscorlib\" xmlns:sdc2=\"clr-namespace:System.Diagnostics.CodeAnalysis;assembly=System\" xmlns:sdc3=\"clr-namespace:System.Data.Common;assembly=System.Data\" xmlns:sdci=\"clr-namespace:System.Diagnostics.Contracts.Internal;assembly=mscorlib\" xmlns:sde=\"clr-namespace:System.Diagnostics.Eventing;assembly=System.Core\" xmlns:sder=\"clr-namespace:System.Diagnostics.Eventing.Reader;assembly=System.Core\" xmlns:sdi=\"clr-namespace:System.Deployment.Internal;assembly=mscorlib\" xmlns:sdo=\"clr-namespace:System.Data.OleDb;assembly=System.Data\" xmlns:sdo1=\"clr-namespace:System.Data.Odbc;assembly=System.Data\" xmlns:sdp=\"clr-namespace:System.Diagnostics.PerformanceData;assembly=System.Core\" xmlns:sds=\"clr-namespace:System.Diagnostics.SymbolStore;assembly=mscorlib\" xmlns:sds1=\"clr-namespace:System.Data.Sql;assembly=System.Data\" xmlns:sds2=\"clr-namespace:System.Data.SqlTypes;assembly=System.Data\" xmlns:sds3=\"clr-namespace:System.Data.SqlClient;assembly=System.Data\" xmlns:sdt=\"clr-namespace:System.Diagnostics.Tracing;assembly=mscorlib\" xmlns:sg=\"clr-namespace:System.Globalization;assembly=mscorlib\" xmlns:si=\"clr-namespace:System.IO;assembly=mscorlib\" xmlns:si1=\"clr-namespace:System.IO;assembly=System.Core\" xmlns:si2=\"clr-namespace:System.IO;assembly=System\" xmlns:si3=\"clr-namespace:System.IO;assembly=System.ServiceModel\" xmlns:sic=\"clr-namespace:System.IO.Compression;assembly=System\" xmlns:sii=\"clr-namespace:System.IO.IsolatedStorage;assembly=mscorlib\" xmlns:sim=\"clr-namespace:System.IO.MemoryMappedFiles;assembly=System.Core\" xmlns:sip=\"clr-namespace:System.IO.Pipes;assembly=System.Core\" xmlns:sip1=\"clr-namespace:System.IO.Ports;assembly=System\" xmlns:sl=\"clr-namespace:System.Linq;assembly=System.Core\" xmlns:sl1=\"clr-namespace:System.Linq;assembly=System.Collections.Immutable\" xmlns:sle=\"clr-namespace:System.Linq.Expressions;assembly=System.Core\" xmlns:sm=\"clr-namespace:System.Media;assembly=System\" xmlns:smi=\"clr-namespace:System.Management.Instrumentation;assembly=System.Core\" xmlns:sn=\"clr-namespace:System.Net;assembly=System\" xmlns:snc=\"clr-namespace:System.Net.Cache;assembly=System\" xmlns:snc1=\"clr-namespace:System.Net.Configuration;assembly=System\" xmlns:snm=\"clr-namespace:System.Net.Mail;assembly=System\" xmlns:snm1=\"clr-namespace:System.Net.Mail;assembly=System\" xmlns:snn=\"clr-namespace:System.Net.NetworkInformation;assembly=System\" xmlns:sns=\"clr-namespace:System.Net.Security;assembly=System\" xmlns:sns1=\"clr-namespace:System.Net.Sockets;assembly=System\" xmlns:snw=\"clr-namespace:System.Net.WebSockets;assembly=System\" xmlns:snw1=\"clr-namespace:System.Net.WebSockets;assembly=System.Net.WebSockets.Client.Managed\" xmlns:snwm=\"clr-namespace:System.Net.WebSockets.Managed;assembly=System.Net.WebSockets.Client.Managed\" xmlns:sr=\"clr-namespace:System.Resources;assembly=mscorlib\" xmlns:sr1=\"clr-namespace:System.Reflection;assembly=mscorlib\" xmlns:sr2=\"clr-namespace:System.Runtime;assembly=mscorlib\" xmlns:sr3=\"clr-namespace:System.Reflection;assembly=System\" xmlns:src=\"clr-namespace:System.Runtime.ConstrainedExecution;assembly=mscorlib\" xmlns:src1=\"clr-namespace:System.Runtime.CompilerServices;assembly=mscorlib\" xmlns:src2=\"clr-namespace:System.Runtime.CompilerServices;assembly=System.Core\" xmlns:src3=\"clr-namespace:System.Runtime.CompilerServices;assembly=Microsoft.Bcl.AsyncInterfaces\" xmlns:src4=\"clr-namespace:System.Runtime.CompilerServices;assembly=System.ValueTuple\" xmlns:srd=\"clr-namespace:System.Runtime.DesignerServices;assembly=mscorlib\" xmlns:sre=\"clr-namespace:System.Reflection.Emit;assembly=mscorlib\" xmlns:sre1=\"clr-namespace:System.Runtime.ExceptionServices;assembly=mscorlib\" xmlns:srh=\"clr-namespace:System.Runtime.Hosting;assembly=mscorlib\" xmlns:sri=\"clr-namespace:System.Runtime.InteropServices;assembly=mscorlib\" xmlns:sri1=\"clr-namespace:System.Runtime.InteropServices;assembly=System.Core\" xmlns:sri2=\"clr-namespace:System.Runtime.InteropServices;assembly=System\" xmlns:sri3=\"clr-namespace:System.Runtime.InteropServices;assembly=System.Memory\" xmlns:sric=\"clr-namespace:System.Runtime.InteropServices.ComTypes;assembly=mscorlib\" xmlns:sric1=\"clr-namespace:System.Runtime.InteropServices.ComTypes;assembly=System\" xmlns:srie=\"clr-namespace:System.Runtime.InteropServices.Expando;assembly=mscorlib\" xmlns:sriw=\"clr-namespace:System.Runtime.InteropServices.WindowsRuntime;assembly=mscorlib\" xmlns:srr=\"clr-namespace:System.Runtime.Remoting;assembly=mscorlib\" xmlns:srra=\"clr-namespace:System.Runtime.Remoting.Activation;assembly=mscorlib\" xmlns:srrc=\"clr-namespace:System.Runtime.Remoting.Contexts;assembly=mscorlib\" xmlns:srrc1=\"clr-namespace:System.Runtime.Remoting.Channels;assembly=mscorlib\" xmlns:srrl=\"clr-namespace:System.Runtime.Remoting.Lifetime;assembly=mscorlib\" xmlns:srrm=\"clr-namespace:System.Runtime.Remoting.Metadata;assembly=mscorlib\" xmlns:srrm1=\"clr-namespace:System.Runtime.Remoting.Messaging;assembly=mscorlib\" xmlns:srrmw=\"clr-namespace:System.Runtime.Remoting.Metadata.W3cXsd2001;assembly=mscorlib\" xmlns:srrp=\"clr-namespace:System.Runtime.Remoting.Proxies;assembly=mscorlib\" xmlns:srrs=\"clr-namespace:System.Runtime.Remoting.Services;assembly=mscorlib\" xmlns:srs=\"clr-namespace:System.Runtime.Serialization;assembly=mscorlib\" xmlns:srs1=\"clr-namespace:System.Runtime.Serialization;assembly=System.Runtime.Serialization\" xmlns:srsc=\"clr-namespace:System.Runtime.Serialization.Configuration;assembly=System.Runtime.Serialization\" xmlns:srsf=\"clr-namespace:System.Runtime.Serialization.Formatters;assembly=mscorlib\" xmlns:srsfb=\"clr-namespace:System.Runtime.Serialization.Formatters.Binary;assembly=mscorlib\" xmlns:srsj=\"clr-namespace:System.Runtime.Serialization.Json;assembly=System.Runtime.Serialization\" xmlns:srv=\"clr-namespace:System.Runtime.Versioning;assembly=mscorlib\" xmlns:srv1=\"clr-namespace:System.Runtime.Versioning;assembly=System\" xmlns:ss=\"clr-namespace:System.Security;assembly=mscorlib\" xmlns:ss1=\"clr-namespace:System.Security;assembly=System.Core\" xmlns:ss2=\"clr-namespace:System.Security;assembly=System\" xmlns:ss3=\"clr-namespace:System.ServiceModel;assembly=System.ServiceModel\" xmlns:ssa=\"clr-namespace:System.Security.AccessControl;assembly=mscorlib\" xmlns:ssa1=\"clr-namespace:System.Security.Authentication;assembly=System\" xmlns:ssa2=\"clr-namespace:System.Security.AccessControl;assembly=System\" xmlns:ssa3=\"clr-namespace:System.ServiceModel.Activation;assembly=System.ServiceModel\" xmlns:ssac=\"clr-namespace:System.ServiceModel.Activation.Configuration;assembly=System.ServiceModel\" xmlns:ssae=\"clr-namespace:System.Security.Authentication.ExtendedProtection;assembly=System\" xmlns:ssaec=\"clr-namespace:System.Security.Authentication.ExtendedProtection.Configuration;assembly=System\" xmlns:ssc=\"clr-namespace:System.Security.Cryptography;assembly=mscorlib\" xmlns:ssc1=\"clr-namespace:System.Security.Claims;assembly=mscorlib\" xmlns:ssc2=\"clr-namespace:System.Security.Cryptography;assembly=System.Core\" xmlns:ssc3=\"clr-namespace:System.Security.Claims;assembly=System\" xmlns:ssc4=\"clr-namespace:System.Security.Cryptography;assembly=System\" xmlns:ssc5=\"clr-namespace:System.ServiceModel.ComIntegration;assembly=System.ServiceModel\" xmlns:ssc6=\"clr-namespace:System.ServiceModel.Configuration;assembly=System.ServiceModel\" xmlns:ssc7=\"clr-namespace:System.ServiceModel.Channels;assembly=System.ServiceModel\" xmlns:sscx=\"clr-namespace:System.Security.Cryptography.X509Certificates;assembly=mscorlib\" xmlns:sscx1=\"clr-namespace:System.Security.Cryptography.X509Certificates;assembly=System.Core\" xmlns:sscx2=\"clr-namespace:System.Security.Cryptography.X509Certificates;assembly=System\" xmlns:ssd=\"clr-namespace:System.ServiceModel.Description;assembly=System.ServiceModel\" xmlns:ssd1=\"clr-namespace:System.ServiceModel.Dispatcher;assembly=System.ServiceModel\" xmlns:ssd2=\"clr-namespace:System.ServiceModel.Diagnostics;assembly=System.ServiceModel\" xmlns:ssm=\"clr-namespace:System.ServiceModel.MsmqIntegration;assembly=System.ServiceModel\" xmlns:ssp=\"clr-namespace:System.Security.Permissions;assembly=mscorlib\" xmlns:ssp1=\"clr-namespace:System.Security.Principal;assembly=mscorlib\" xmlns:ssp2=\"clr-namespace:System.Security.Policy;assembly=mscorlib\" xmlns:ssp3=\"clr-namespace:System.Security.Permissions;assembly=System\" xmlns:ssp4=\"clr-namespace:System.ServiceModel.PeerResolvers;assembly=System.ServiceModel\" xmlns:sss=\"clr-namespace:System.ServiceModel.Syndication;assembly=System.ServiceModel\" xmlns:sss1=\"clr-namespace:System.ServiceModel.Security;assembly=System.ServiceModel\" xmlns:ssst=\"clr-namespace:System.ServiceModel.Security.Tokens;assembly=System.ServiceModel\" xmlns:ssx=\"clr-namespace:System.ServiceModel.XamlIntegration;assembly=System.ServiceModel\" xmlns:st=\"clr-namespace:System.Threading;assembly=mscorlib\" xmlns:st1=\"clr-namespace:System.Text;assembly=mscorlib\" xmlns:st2=\"clr-namespace:System.Threading;assembly=System.Core\" xmlns:st3=\"clr-namespace:System.Timers;assembly=System\" xmlns:st4=\"clr-namespace:System.Threading;assembly=System\" xmlns:str=\"clr-namespace:System.Text.RegularExpressions;assembly=System\" xmlns:stt=\"clr-namespace:System.Threading.Tasks;assembly=mscorlib\" xmlns:stt1=\"clr-namespace:System.Threading.Tasks;assembly=System.Core\" xmlns:stt2=\"clr-namespace:System.Threading.Tasks;assembly=Microsoft.Bcl.AsyncInterfaces\" xmlns:stts=\"clr-namespace:System.Threading.Tasks.Sources;assembly=Microsoft.Bcl.AsyncInterfaces\" xmlns:sw=\"clr-namespace:System.Web;assembly=System\" xmlns:swi=\"clr-namespace:System.Windows.Input;assembly=System\" xmlns:swm=\"clr-namespace:System.Web.Mail;assembly=System.Web\" xmlns:sx=\"clr-namespace:System.Xml;assembly=System.Xml\" xmlns:sx1=\"clr-namespace:System.Xml;assembly=System.Runtime.Serialization\" xmlns:sx2=\"clr-namespace:System.Xml;assembly=System.Data\" xmlns:sxr=\"clr-namespace:System.Xml.Resolvers;assembly=System.Xml\" xmlns:sxs=\"clr-namespace:System.Xml.Serialization;assembly=System.Xml\" xmlns:sxs1=\"clr-namespace:System.Xml.Schema;assembly=System.Xml\" xmlns:sxsa=\"clr-namespace:System.Xml.Serialization.Advanced;assembly=System.Xml\" xmlns:sxsc=\"clr-namespace:System.Xml.Serialization.Configuration;assembly=System.Xml\" xmlns:sxx=\"clr-namespace:System.Xml.Xsl;assembly=System.Xml\" xmlns:sxx1=\"clr-namespace:System.Xml.XPath;assembly=System.Xml\" xmlns:sxx2=\"clr-namespace:System.Xml.XmlConfiguration;assembly=System.Xml\" xmlns:x=\"http://schemas.microsoft.com/winfx/2006/xaml\"><TextExpression.NamespacesForImplementation><sco:Collection x:TypeArguments=\"x:String\"><x:String>System</x:String><x:String>System.Xml</x:String><x:String>beBOT</x:String><x:String>System.Data</x:String><x:String>System.Linq</x:String><x:String>Microsoft.VisualBasic</x:String><x:String>beBOT.Image</x:String></sco:Collection></TextExpression.NamespacesForImplementation><TextExpression.ReferencesForImplementation><sco:Collection x:TypeArguments=\"AssemblyReference\"><AssemblyReference>mscorlib</AssemblyReference><AssemblyReference>System.Xml</AssemblyReference><AssemblyReference>beBOT</AssemblyReference><AssemblyReference>beBOT.Interfaces</AssemblyReference><AssemblyReference>System.Data</AssemblyReference><AssemblyReference>System.Core</AssemblyReference><AssemblyReference>Microsoft.VisualBasic</AssemblyReference><AssemblyReference>System.Data.DataSetExtensions</AssemblyReference><AssemblyReference>System</AssemblyReference><AssemblyReference>System.Memory</AssemblyReference><AssemblyReference>ClosedXML</AssemblyReference><AssemblyReference>Microsoft.Bcl.AsyncInterfaces</AssemblyReference><AssemblyReference>Microsoft.Bcl.HashCode</AssemblyReference><AssemblyReference>System.ServiceModel</AssemblyReference><AssemblyReference>System.ComponentModel.Composition</AssemblyReference><AssemblyReference>System.Net.WebSockets.Client.Managed</AssemblyReference><AssemblyReference>System.Runtime.WindowsRuntime</AssemblyReference><AssemblyReference>beBOT.Image</AssemblyReference><AssemblyReference>System.Activities</AssemblyReference></sco:Collection></TextExpression.ReferencesForImplementation>{custom_code}<sap2010:WorkflowViewState.ViewStateManager><sap2010:ViewStateManager><sap2010:ViewStateData Id=\"Sequence_2\" sap:VirtualizedContainerService.HintSize=\"222,175\" /><sap2010:ViewStateData Id=\"OpenBrowser_1\" sap:VirtualizedContainerService.HintSize=\"200,55\" /><sap2010:ViewStateData Id=\"Sequence_3\" sap:VirtualizedContainerService.HintSize=\"222,175\" /><sap2010:ViewStateData Id=\"Sequence_1\" sap:VirtualizedContainerService.HintSize=\"222,175\" /><sap2010:ViewStateData Id=\"Migration\" sap:VirtualizedContainerService.HintSize=\"262,316\" /></sap2010:ViewStateManager></sap2010:WorkflowViewState.ViewStateManager></Activity>",
  "culture": "ko-KR",
  "Serializable": true,
  "background": false,
  "Filename": "openurl_uipath.json",
  "projectandname": "Migration/openurl_uipath.json",
  "FilePath": "C:\\Users\\UserK\\Desktop\\서연진_2024_하계실습\\Migration_테스트\\openurl_uipath.json",
  "projectid": "53e36696 - 31d2 - 4baf - 9bba - 873bb4dff2fd",
  "name": "openurl_uipath.json",
  "_id": "9b24bc9d - 6da6 - 47c8 - 9095 - 4c853a0ad714",
  "_type": "workflow",
  "_modifiedby": null,
  "_modifiedbyid": null,
  "_created": "0001 - 01 - 01T00: 00:00",
  "_createdby": null,
  "_createdbyid": null,
  "_acl": null,
  "_encrypt": null,
  "_version": 3
}}
"""

final_output = template.format(custom_code=escaped_xml)

print(final_output)

{
  "queue": null,
  "Xaml": "<Activity mc1:Ignorable=\"sads sap sap2010\" x:Class=\"openurl_uipath.xaml\" sap2010:WorkflowViewState.IdRef=\"새_워크플로우_1\" mva1:VisualBasic.Settings=\"Assembly references and imported namespaces serialized as XML namespaces\" xmlns=\"http://schemas.microsoft.com/netfx/2009/xaml/activities\" xmlns:b=\"clr-namespace:beBOT;assembly=beBOT\" xmlns:b1=\"clr-namespace:beBOT;assembly=beBOT.Interfaces\" xmlns:ba=\"clr-namespace:beBOT.Activities;assembly=beBOT\" xmlns:bi=\"clr-namespace:beBOT.Image;assembly=beBOT.Image\" xmlns:bd=\"clr-namespace:beBOT.Database;assembly=beBOT.Database\" xmlns:bi1=\"clr-namespace:beBOT.Interfaces;assembly=beBOT.Interfaces\" xmlns:bi2=\"clr-namespace:beBOT.Interfaces;assembly=beBOT.Interfaces\" xmlns:bia=\"clr-namespace:beBOT.Interfaces.Activities;assembly=beBOT.Interfaces\" xmlns:bie=\"clr-namespace:beBOT.Interfaces.entity;assembly=beBOT.Interfaces\" xmlns:bii=\"clr-namespace:beBOT.Interfaces.IPCService;assembly=beBOT.Interfaces\" xml

In [ ]:

thread = client.beta.threads.create()


def gpt_llm(query):
  message = client.beta.threads.messages.create(
      thread_id = thread.id,
      role = "user",
      content= "Please connect me to naver",
  )
  return message

run = client.beta.threads.runs.create_and_poll(
    thread_id = thread.id,
    assistant_id = assistant.id,
)
show_json(run)

import gradio as gr
from bs4 import BeautifulSoup as bs
import pprint
from halo import Halo

def generate_response(messages):
    # Create a loading spinner
    spinner = Halo(text='Loading...', spinner='dots')
    spinner.start()

# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return gpt_llm(question, formatted_context)

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
  outputs="text",
  title="RAG with Llama3",
  description="Ask questions about the proveded context",
)

# Launch the Gradio app
iface.launch()


In [ ]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  show_json(messages)
  print_message(thread.id)
else:
  print(run.status)

## 실행 단계를 살펴보기 위해 Steps 활용
- 질문에 대해 답을 만들 때 file search를 사용한 것을 확인할 수 있습니다.

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id, run_id = run.id, order="asc",
)
show_json(run_steps)

## Reference
- [File Search - Assistants API(Beta) - Open AI](https://platform.openai.com/docs/assistants/tools/file-search)
- [EP02. #openai의 새로운 기능 #assistant API 3가지 도구 활용법 - 테디노트](https://www.youtube.com/watch?v=BMW1NJkL7Ks)
- [소스 코드 참조 - Colab](https://colab.research.google.com/drive/1bGtWYX5TQ3uqzEMFdzTGW9JUD-iMB6O2?usp=sharing)